In [1]:
# Importing libraries
import nltk
from nltk import FreqDist
import pandas as pd
import numpy as np
import re
import gensim
from gensim import corpora
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import seaborn as sns

In [2]:
# findinf frequence of words
def freq_words(x, terms = 30):
    all_words = ' '.join([str(text) for text in x])
    all_words = all_words.split()
  
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()), 'count':list(fdist.values())})
    
    # selecting top 20 most frequent words
    d = words_df.nlargest(columns="count", n = terms) 
    plt.figure(figsize=(20,5))
    ax = sns.barplot(data=d, x= "word", y = "count")
    ax.set(ylabel = 'Count')
    plt.show()


In [3]:
# function to remove stopwords
def remove_stopwords(rev):
    rev_new = " ".join([i for i in rev if i not in stopwords.words('english')])
    return rev_new

In [4]:
# user porterstemmer for stemming
def stemming(texts): 
    s_stemmer = PorterStemmer()
    for i in range(len(texts)):
        texts[i]=[s_stemmer.stem(texts[i][j]) for j in range(len(texts[i]))]
        return texts

In [5]:
# Read the csv file
df = pd.read_csv('reviews2.csv')


# replace "n't" with " not"
df['Review'] = df['Review'].str.replace("n\'t", " not")

# remove unwanted characters, numbers and symbols
df['Review'] = df['Review'].str.replace("[^a-zA-Z#]", " ")

# remove short words (length < 3)
df['Review'] = df['Review'].apply(lambda x: ' '.join([str(w) for w in str(x).split() if len(w)>2]))

In [6]:
# remove stopwords from the text
reviews = [remove_stopwords(r.split()) for r in df['Review']]

# make entire text lowercase
reviews = [r.lower() for r in reviews]
        
tokenized_reviews = pd.Series(reviews).apply(lambda x: x.split())
reviews_2 = stemming(tokenized_reviews)

In [7]:
# Create the term dictionary of our corpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(reviews_2)#reviews_2)

# Convert list of reviews (reviews_2) into a Document Term Matrix using the dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(rev) for rev in reviews_2]#reviews_2]
   

In [ ]:
reviews# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix,
                id2word=dictionary,
                num_topics=7, 
                random_state=100,
                chunksize=1000,
                passes=50)

lda_model.print_topics()

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary)
vis

# Print the Keyword in the 10 topics
lda_model.print_topics() 